In [1]:
import aco as aco
import random 
import numpy as np
import threading
import time

import prog_lineraire as pl
import generator_matrix as gm
import dataset
import json
import pandas as pd

In [2]:
def generate_nodes(number,max_x=1080,max_y=720,seed=None):
    # create number of nodes with coordinates of x,y  with max value of 600
    random.seed(seed)
    nodes = []
    for i in range(number):
        nodes.append([random.randint(0.1*max_y,max_x-0.1*max_y),random.randint(0.1*max_y,max_y-0.1*max_y)])
    return nodes

def distance(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def coordinats_to_matrice(nodes):
    # create a matrice with the nodes coordinates
    matrice = np.zeros((len(nodes),len(nodes)))
    for x in range(len(nodes)):
        for y in range(len(nodes)):
            matrice[x][y] = distance(nodes[x][0],nodes[x][1],nodes[y][0],nodes[y][1])
    return matrice

In [3]:
#
#nbr_iter=100
#nbr_ant=10
#a_pheromone = 1
#b_heuristic = 1
#p_evaporate = 0.1
#size = 10
#seed= 10
#
#matrice_pondere = coordinats_to_matrice(generate_nodes(10,seed=seed))
#
#pheromone,best_path,best_score,time_record = aco.aco_algorithm(matrice_pondere,nbr_iter,nbr_ant,a_pheromone,b_heuristic,p_evaporate,seed)
#
#

In [4]:

parameters = {
    "nbr_ant":np.arange(25,500,25),
    "a":np.arange(0,4.25,.25),
    "b":np.arange(1,4.25,.25),
    "p":np.arange(0,1.05,.05),
    "nb_iter_max":[1000],
    "time_max":[120],
    "nbr_random_search":[10],
    "nbr_loops":[5],
}
#nbr_dataset = 1
#nbr_sommet = 50


matrice= dataset.import_dataset("dataset/small_dataset")
villes = dataset.import_dataset("dataset/small_cities")

#datasets = generate_datasets(nbr_dataset,nbr_sommet,seed=None)
datasets = [matrice]
borne_inf = pl.prog_lineaire(matrice, villes)[-1]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/colin/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/77f35d549f9a4e5592b7655e98d91cb4-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/77f35d549f9a4e5592b7655e98d91cb4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 80 COLUMNS
At line 3206 RHS
At line 3282 BOUNDS
At line 3908 ENDATA
Problem MODEL has 75 rows, 625 columns and 1275 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3999.64 - 0.00 seconds
Cgl0002I 25 variables fixed
Cgl0004I processed model has 50 rows, 600 columns (600 integer (600 of which binary)) and 1200 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 3999.64
Cbc0038I Before mini branch and bound, 600 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did

In [5]:
borne_inf

4433.9271222971

# Optimisation
Comment evaluer mes algorithmes ?
Comment faire pour comparer les algorithmes ?

Déterminer la borne pour chacun des dataset

Tester une combiaison de paramètres sur chaque dataset pour obtenir une moyenne et l'ecart type:


1. Prendre une combinaison de paramètres
2. Tester les paramètres sur chaque dataset 10 fois.
   1. Récupérer la moyenne et l'écart type des 10 essais sur chaque dataset.
1. Evaluer l'effcatité d'une combinaison.
   






In [6]:
def generate_datasets(nbr_dataset,nbr_sommet,seed=None):
    """Generate nbr_dataset datasets with nbr_sommet nodes

    Args:
        nbr_dataset (_type_): _description_
        nbr_sommet (_type_): _description_
        seed (_type_, optional): _description_. Defaults to None.

    Returns:
        array: array of datasets
    """

    datasets = []
    for i in range(nbr_dataset):
        datasets.append(coordinats_to_matrice(generate_nodes(nbr_sommet,seed=None)))
    return datasets

def random_parameters(parameters):
    """Return random parameters in parameters

    Args:
        parameters (array): array of parameters

    Returns:
        dict: return dict of random parameters
    """
    return {key:random.choice(parameters[key]) for key in parameters}


In [7]:
#df =   pd.DataFrame(columns=["nbr_ant","a","b","p","nb_iter_max","time_max","nbr_random_search","nbr_loops","mean_score","std_score"])
#df.to_json("dataset/results_rdm_search_cv.json")

In [8]:
def GridSearchCV(parameters,datasets):


    for rdm_search in range(parameters["nbr_random_search"][0]):
        
        parameter = random_parameters(parameters)

        print("Random search : {}/{}".format(rdm_search+1,parameters["nbr_random_search"][0]))
        print("Parameters : {}".format(parameter))

        mean_scores = []
        standard_deviations = []


        for dataset in datasets:
            #
            

            mean_score = 0
            aco_result_list = []

            thrd_list = []
            start = time.time()

            for loop in range(parameters["nbr_loops"][0]):
                thread = threading.Thread(target=aco.aco_algorithm, args=(dataset,parameter,aco_result_list,))
                thrd_list.append(thread)
                thread.start()
                
                
            for thread in thrd_list:  # iterates over the threads
                #difference de temps avec start
                
                thread.join()     
                #print("Temps d'execution: " + str(time.time() - start))
            


            # pourcentage de différence entre la programation linéaire et l'aco

            mean_score = np.mean([borne_inf*100/(aco.path_distance(aco_result_list[i][1],matrice)) for i in range(len(aco_result_list))])
            standard_deviation = np.std([borne_inf*100/(aco.path_distance(aco_result_list[i][1],matrice)) for i in range(len(aco_result_list))])

            df = pd.read_json("dataset/results_rdm_search_cv.json")
            row = {"nbr_ant":parameter["nbr_ant"],"a":parameter["a"],"b":parameter["b"],"p":parameter["p"],"nb_iter_max":parameter["nb_iter_max"],"time_max":parameter["time_max"],"nbr_random_search":parameter["nbr_random_search"],"nbr_loops":parameter["nbr_loops"],"score":mean_score,"std_score":standard_deviation}
            df = df.append(row,ignore_index=True)
            df.to_json("dataset/results_rdm_search_cv.json")

        


In [9]:

#GridSearchCV(parameters,datasets)

In [10]:
df = pd.read_json("dataset/results_rdm_search_cv.json").sort_values(by="score",ascending=False)
print(df)

    nbr_ant     a     b     p  nb_iter_max  time_max  nbr_random_search  \
12      200  1.75  3.00  0.90         1000       120                 10   
19      475  1.25  1.75  0.95         1000       120                 10   
1       200  1.75  4.00  0.10         1000       120                 10   
8       450  1.50  4.00  0.85         1000       120                 10   
3       200  1.50  3.25  0.05         1000       120                 10   
2       150  0.75  3.75  0.25         1000       120                 10   
18      475  1.50  2.50  0.20         1000       120                 10   
10      250  0.75  3.00  0.15         1000       120                 10   
5       175  2.25  3.50  0.65         1000       120                 10   
0       450  2.25  3.75  0.50         1000       120                 10   
9       150  3.75  2.00  0.70         1000       120                 10   
11      200  3.50  2.00  0.25         1000       120                 10   
13      475  2.75  1.50  

In [12]:

contrains = {
    "start":3,
    "city":1,
    "inferior_dist":7,
    "superior_dist":10,
}
res = []
aco.aco_algorithm(matrice,parameters,res,contrains)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6]
[1, 2,

ValueError: 1 is not in list